In [1]:
from zipfile import ZipFile
file_name = 'drive/MyDrive/SDP_Datasets/brain-tumor.zip'

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [4]:
train_path = 'Training'
valid_path = 'Testing'

In [5]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
# IMAGE_SIZE + [3] - RGB images
# include_top = False removes last layer of VGG16 since VGG16 classifies 1000 category of images

58892288/58889256 [==============================] - 0s 0us/step


In [6]:
# Model is already trained, don't train existing weights again
for layer in vgg.layers:
    layer.trainable = False

In [7]:
# useful for getting number of classes of dataset
folders = glob('Training/*')

In [8]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x) # our layers - you can add more if you want
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [10]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [13]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs = 20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
90/90 [==============================] - 49s 456ms/step - loss: 1.4143 - accuracy: 0.5305 - val_loss: 1.7969 - val_accuracy: 0.5660
Epoch 2/20
90/90 [==============================] - 39s 437ms/step - loss: 0.4602 - accuracy: 0.8296 - val_loss: 1.5231 - val_accuracy: 0.6878
Epoch 3/20
90/90 [==============================] - 40s 439ms/step - loss: 0.3525 - accuracy: 0.8567 - val_loss: 1.5723 - val_accuracy: 0.6878
Epoch 4/20
90/90 [==============================] - 40s 439ms/step - loss: 0.2835 - accuracy: 0.9013 - val_loss: 1.2539 - val_accuracy: 0.6929
Epoch 5/20
90/90 [==============================] - 40s 441ms/step - loss: 0.2659 - accuracy: 0.8982 - val_loss: 1.5205 - val_accuracy: 0.7335
Epoch 6/20
90/90 [==============================] - 39s 438ms/step - loss: 0.2256 - accuracy: 0.9217 - val_loss: 1.5595 - val_accuracy: 0.7411
Epoch 7/20
90/90 [==============================] - 40s 440ms/step - loss: 0.1935 - accuracy: 0.9248 - val_loss: 1.7764 - val_accuracy: 0.7360

In [14]:
from keras.models import load_model
model.save('brain-tumor-model.h5')